In [1]:
# Iván Andrés Trujillo Abella
# ivantrujillo1229@gmail.com

In [3]:
import pandas as pd

# Solving XOR

The following notebook is a simple neural network with one layer.

You can find more information avaiable on:


In [26]:
df = pd.DataFrame([[0,0,0], [0,1,1], [1,0,1], [1,1,0]],
                  columns = ['x1', 'x2','y'])

In [11]:
import numpy as np
import pandas as pd
from numpy.random import randn
from numpy import exp
from numpy import array
from numpy import vectorize
from numpy import append
from numpy import log
from numpy import clip

In [16]:
def sigmoid(x):
  return 1 / (1 + exp(-x))
sigmoid = vectorize(sigmoid)  ## Acts similar to map (func, iter)

In [19]:
## J(w) cost function
def BinaryLog(y_true, est_probabilities):
  # Pass in bothe vectores in narrays
  est_probabilities = clip(est_probabilities, 0.00000000000001, 0.99999999999999) # not defined estimated probs in zero or one
  return -((y_true * log(est_probabilities) + (1 - y_true) * log(1-est_probabilities)).sum())/y_true.shape[0]
  # log_loss have a better implementation!

def extractW(solution, hidden_size, input_size):
  w1 = solution[0:hidden_size*input_size].reshape((hidden_size, input_size))
  w2 = solution[hidden_size*input_size:]
  return w1, w2

def joinW(w1,w2):
  W = append(w1,w2).reshape(-1)
  return W

def predict(solution, xi, hidden_size, input_size):
    W1,W2 = extractW(solution, hidden_size, input_size)
    z1 = sigmoid( W1 @ xi) # The first computation
    output = sigmoid(W2 @ z1)
    return output

def MLP_fitnessFunction(hidden_size,  df, X, target, initial_solution=0, preds=False):
  input_size = len(X)
  X = df[X].copy()
  if len(initial_solution)==0:
    P = (hidden_size * input_size) + hidden_size
    solution = randn(P)
  else:
    solution = array(initial_solution) # Consider this to keep list
  y_probs = np.zeros(df.shape[0])  # Array to keep the solutions...
  for index, xi in enumerate(X.to_numpy()):
    y_probs[index] = predict(solution, xi, hidden_size, input_size)
  if preds==True:
    return y_probs
  else:
    return BinaryLog(df[target].to_numpy(), y_probs),


In [33]:
## Testing Forward
X  = ['x1', 'x2']
hidden_size = 3
input_size = len(X)
P = (hidden_size * input_size) + hidden_size
solution = randn(P)
solution = list(solution)
MLP_fitnessFunction(hidden_size, df=df, X=X, target='y', initial_solution=solution)

(0.684101479545901,)

In [30]:
!pip install deap > null

In [31]:
import random
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [38]:
hidden_size = 5 # Neurons in hidden layer
input_size = len(X)
individual_size = (input_size * hidden_size) + hidden_size
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
# Attribute generator
toolbox.register("geneNormal", random.gauss, 0, 10)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.geneNormal, individual_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", lambda chromosome: MLP_fitnessFunction(hidden_size,
                                                                    X=X, target='y', df=df, initial_solution=chromosome ))
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=10 , indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=10)



def main():
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, logBook = algorithms.eaSimple(pop, toolbox, cxpb=0.85, mutpb=0.05, ngen=35,
                                   stats=stats, halloffame=hof, verbose=True)
    return hof, logBook
best, logBook = main()
#best[0].fitness

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	std    	min    	max    
0  	50    	5.69475	2.94238	1.20966	13.3739
1  	46    	2.8815 	2.4394 	0.287041	12.4619
2  	40    	0.963417	0.772775	0.25256 	3.60739
3  	42    	0.335587	0.181987	0.25256 	1.20181
4  	42    	0.41023 	0.832933	0.25256 	6.16288
5  	44    	0.293197	0.257771	0.250152	2.08693
6  	41    	0.403994	0.707561	0.250152	4.79315
7  	44    	0.255059	0.0282245	0.250152	0.452567
8  	34    	0.302945	0.369087 	0.250152	2.88655 
9  	48    	0.250184	0.000228787	0.250152	0.251786
10 	45    	0.254888	0.0331556  	0.250152	0.486977
11 	40    	0.369703	0.836861   	0.250152	6.22773 
12 	42    	0.26808 	0.125535   	0.249909	1.14683 
13 	44    	0.361518	0.761257   	0.249909	5.68875 
14 	46    	0.252498	0.0177046  	0.249909	0.376428
15 	41    	0.319827	0.294454   	0.249893	1.97179 
16 	36    	0.297052	0.251646   	0.249893	1.92807 
17 	45    	0.267155	0.0938125  	0.249893	0.884547
18 	43    	0.304531	0.382466   	0.249893	2.98179 
19 	43    	0.25899 	0.0636602  	0.249893	0.7

In [43]:
SOLUTION = best[0]
df['GA_preds'] = np.where(MLP_fitnessFunction(hidden_size=hidden_size,  X=X, target='y',
                                       df=df,  initial_solution=SOLUTION, preds=True)>0.5,1,0)

In [45]:
df # Solved!

,x1,x2,y,GA_preds
0,0,0,0,0
1,0,1,1,1
2,1,0,1,1
3,1,1,0,0
